<a href="https://colab.research.google.com/github/vsbca/Data-Science/blob/master/Pyspark_training_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import random 
import numpy as np

#https://github.com/MLWhiz/data_science_blogs/blob/master/sparkdf/Working%20With%20Spark.ipynb

Colab notebook, need to install pyspark

In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 51 kB/s 
     |████████████████████████████████| 199 kB 65.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=2d6fa23a2caaf2017a6efb64231fe389a64ba4585e39958576ecf65a17734f12
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [4]:
#Importing pyspark now
import pyspark
from pyspark.sql import SparkSession

In [5]:
#Create SparkSession
spark = SparkSession.builder.master("local[1]")\
                    .appName("sparktutorial")\
                    .getOrCreate()

In [9]:
!wget https://raw.githubusercontent.com/vsbca/Data-Science/master/Data/coronacases.csv -o coronacases.csv

In [10]:
!ls -l

total 16
-rw-r--r-- 1 root root  601 Nov  5 01:38 coronacases.csv
-rw-r--r-- 1 root root 7295 Nov  5 01:38 coronacases.csv.1
drwxr-xr-x 1 root root 4096 Nov  3 13:31 sample_data


In [11]:
#Load the data from Git
df = spark.read.load("coronacases.csv.1", format="csv", sep=",", inferSchema="true", header="true")


In [12]:
df.show()

+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|   Seoul|        Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|   Seoul|  Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|   Seoul|   Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|   Seoul|   Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000007|   Seoul|      Jongno-gu| true|Jongno Community ...|       10| 37.57681|   127.006|
|1000008|   Seoul|        Jung-gu| true|Jung-gu Fashion C...|        7

In [13]:
#Convert to Pandas
df.limit(100).toPandas()


,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,72,37.538621,126.992652
1,1000002,Seoul,Guro-gu,True,Guro-gu Call Center,98,37.508163,126.884387
2,1000003,Seoul,Dongdaemun-gu,True,Dongan Church,20,37.592888,127.056766
3,1000004,Seoul,Guro-gu,True,Manmin Central Church,41,37.481059,126.894343
4,1000005,Seoul,Eunpyeong-gu,True,Eunpyeong St. Mary's Hospital,14,37.63369,126.9165
...,...,...,...,...,...,...,...,...
95,6000007,Gyeongsangbuk-do,Yechun-gun,True,Yechun-gun,40,36.646845,128.437416
96,6000008,Gyeongsangbuk-do,Gyeongsan-si,True,Gyeongsan Jeil Silver Town,17,35.84819,128.7621
97,6000009,Gyeongsangbuk-do,Gyeongsan-si,True,Gyeongsan Cham Joeun Community Center,16,35.82558,128.7373
98,6000010,Gyeongsangbuk-do,-,False,overseas inflow,10,-,-


#Change column names

In [14]:
df = df.withColumnRenamed("province", "state")

In [15]:
df.show()

+-------+-----+---------------+-----+--------------------+---------+---------+----------+
|case_id|state|           city|group|      infection_case|confirmed| latitude| longitude|
+-------+-----+---------------+-----+--------------------+---------+---------+----------+
|1000001|Seoul|     Yongsan-gu| true|       Itaewon Clubs|       72|37.538621|126.992652|
|1000002|Seoul|        Guro-gu| true| Guro-gu Call Center|       98|37.508163|126.884387|
|1000003|Seoul|  Dongdaemun-gu| true|       Dongan Church|       20|37.592888|127.056766|
|1000004|Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000005|Seoul|   Eunpyeong-gu| true|Eunpyeong St. Mar...|       14| 37.63369|  126.9165|
|1000006|Seoul|   Seongdong-gu| true|    Seongdong-gu APT|       13| 37.55713|  127.0403|
|1000007|Seoul|      Jongno-gu| true|Jongno Community ...|       10| 37.57681|   127.006|
|1000008|Seoul|        Jung-gu| true|Jung-gu Fashion C...|        7|37.562405|126.984377|
|1000009|S

In [16]:
df = df.select('state', 'city', 'infection_case', 'confirmed')
df.show()

+-----+---------------+--------------------+---------+
|state|           city|      infection_case|confirmed|
+-----+---------------+--------------------+---------+
|Seoul|     Yongsan-gu|       Itaewon Clubs|       72|
|Seoul|        Guro-gu| Guro-gu Call Center|       98|
|Seoul|  Dongdaemun-gu|       Dongan Church|       20|
|Seoul|        Guro-gu|Manmin Central Ch...|       41|
|Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|Seoul|      Jongno-gu|Jongno Community ...|       10|
|Seoul|        Jung-gu|Jung-gu Fashion C...|        7|
|Seoul|from other city|  Shincheonji Church|        8|
|Seoul|              -|     overseas inflow|      321|
|Seoul|              -|contact with patient|       18|
|Seoul|              -|                 etc|       24|
|Busan|     Dongnae-gu|       Onchun Church|       39|
|Busan|from other city|  Shincheonji Church|       12|
|Busan|     Suyeong-gu|Suyeong-gu Kinder...|        5|
|Busan|   

In [17]:
df.sort("confirmed").show()

+-----------------+---------------+--------------------+---------+
|            state|           city|      infection_case|confirmed|
+-----------------+---------------+--------------------+---------+
|          Jeju-do|              -|contact with patient|        0|
|       Gangwon-do|              -|contact with patient|        0|
|          Gwangju|              -|                 etc|        0|
|            Busan|from other city|Cheongdo Daenam H...|        1|
|          Gwangju|              -|contact with patient|        1|
|          Jeju-do|from other city|       Itaewon Clubs|        1|
|           Sejong|from other city|  Shincheonji Church|        1|
|           Sejong|              -|                 etc|        1|
|Chungcheongnam-do|              -|contact with patient|        1|
|     Jeollabuk-do|from other city|  Shincheonji Church|        1|
|     Jeollanam-do|from other city|  Shincheonji Church|        1|
|          Incheon|from other city|  Shincheonji Church|      

In [18]:
#descending sort
from pyspark.sql import functions as F
df.sort(F.desc("confirmed")).show()

+-----------------+---------------+--------------------+---------+
|            state|           city|      infection_case|confirmed|
+-----------------+---------------+--------------------+---------+
|            Daegu|         Nam-gu|  Shincheonji Church|     4510|
|            Daegu|              -|contact with patient|      929|
|            Daegu|              -|                 etc|      724|
| Gyeongsangbuk-do|from other city|  Shincheonji Church|      566|
|            Seoul|              -|     overseas inflow|      321|
|      Gyeonggi-do|              -|     overseas inflow|      225|
|            Daegu|   Dalseong-gun|Second Mi-Ju Hosp...|      196|
| Gyeongsangbuk-do|              -|contact with patient|      192|
| Gyeongsangbuk-do|              -|                 etc|      134|
|            Daegu|         Seo-gu|Hansarang Convale...|      128|
| Gyeongsangbuk-do|   Cheongdo-gun|Cheongdo Daenam H...|      120|
|Chungcheongnam-do|     Cheonan-si|gym facility in C...|      

In [19]:
df.dtypes

[('state', 'string'),
 ('city', 'string'),
 ('infection_case', 'string'),
 ('confirmed', 'int')]

##Cast

In [20]:
from pyspark.sql.types import IntegerType, StringType, DoubleType
df = df.withColumn('confirmed', F.col('confirmed').cast(IntegerType()))

df = df.withColumn('city', F.col('city').cast(StringType()))
df.dtypes

[('state', 'string'),
 ('city', 'string'),
 ('infection_case', 'string'),
 ('confirmed', 'int')]

##Filter

In [21]:
df.filter((df.confirmed > 10) & (df.state == "Gyeongsangbuk-do")).show()

+----------------+---------------+--------------------+---------+
|           state|           city|      infection_case|confirmed|
+----------------+---------------+--------------------+---------+
|Gyeongsangbuk-do|from other city|  Shincheonji Church|      566|
|Gyeongsangbuk-do|   Cheongdo-gun|Cheongdo Daenam H...|      120|
|Gyeongsangbuk-do|    Bonghwa-gun|Bonghwa Pureun Nu...|       68|
|Gyeongsangbuk-do|from other city|Pilgrimage to Israel|       49|
|Gyeongsangbuk-do|    Chilgok-gun|       Milal Shelter|       36|
|Gyeongsangbuk-do|   Gyeongsan-si|Gyeongsan Seorin ...|       66|
|Gyeongsangbuk-do|     Yechun-gun|          Yechun-gun|       40|
|Gyeongsangbuk-do|   Gyeongsan-si|Gyeongsan Jeil Si...|       17|
|Gyeongsangbuk-do|   Gyeongsan-si|Gyeongsan Cham Jo...|       16|
|Gyeongsangbuk-do|              -|contact with patient|      192|
|Gyeongsangbuk-do|              -|                 etc|      134|
+----------------+---------------+--------------------+---------+



#GroupBy

In [22]:
from pyspark.sql import functions as F
df.groupBy(['state', 'city']).agg(F.sum("confirmed"), F.max("confirmed")).show()


+----------------+---------------+--------------+--------------+
|           state|           city|sum(confirmed)|max(confirmed)|
+----------------+---------------+--------------+--------------+
|Gyeongsangnam-do|       Jinju-si|            10|            10|
|           Seoul|        Guro-gu|           139|            98|
|         Daejeon|              -|            27|            10|
|    Jeollabuk-do|from other city|             1|             1|
|Gyeongsangnam-do|Changnyeong-gun|             7|             7|
|           Seoul|              -|           363|           321|
|         Jeju-do|from other city|             1|             1|
|Gyeongsangbuk-do|              -|           336|           192|
|Gyeongsangnam-do|   Geochang-gun|            18|            10|
|         Incheon|from other city|            22|            20|
|           Busan|              -|            72|            29|
|           Daegu|         Seo-gu|           128|           128|
|           Busan|     Su

In [25]:
#Change the name of the column in the GroupBy
df.groupBy(['state', 'city']).agg(F.sum("confirmed").alias("TotalConfirmed"), F.max("confirmed")).show()

+----------------+---------------+--------------+--------------+
|           state|           city|TotalConfirmed|max(confirmed)|
+----------------+---------------+--------------+--------------+
|Gyeongsangnam-do|       Jinju-si|            10|            10|
|           Seoul|        Guro-gu|           139|            98|
|         Daejeon|              -|            27|            10|
|    Jeollabuk-do|from other city|             1|             1|
|Gyeongsangnam-do|Changnyeong-gun|             7|             7|
|           Seoul|              -|           363|           321|
|         Jeju-do|from other city|             1|             1|
|Gyeongsangbuk-do|              -|           336|           192|
|Gyeongsangnam-do|   Geochang-gun|            18|            10|
|         Incheon|from other city|            22|            20|
|           Busan|              -|            72|            29|
|           Daegu|         Seo-gu|           128|           128|
|           Busan|     Su

In [26]:
df.groupBy(["state","city"]).agg(
    F.sum("confirmed").alias("TotalConfirmed"),\
    F.max("confirmed").alias("MaxFromOneConfirmedCase")\
    ).show()

+----------------+---------------+--------------+-----------------------+
|           state|           city|TotalConfirmed|MaxFromOneConfirmedCase|
+----------------+---------------+--------------+-----------------------+
|Gyeongsangnam-do|       Jinju-si|            10|                     10|
|           Seoul|        Guro-gu|           139|                     98|
|         Daejeon|              -|            27|                     10|
|    Jeollabuk-do|from other city|             1|                      1|
|Gyeongsangnam-do|Changnyeong-gun|             7|                      7|
|           Seoul|              -|           363|                    321|
|         Jeju-do|from other city|             1|                      1|
|Gyeongsangbuk-do|              -|           336|                    192|
|Gyeongsangnam-do|   Geochang-gun|            18|                     10|
|         Incheon|from other city|            22|                     20|
|           Busan|              -|    